In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
import re

In [6]:
#URLS to scrap every friday

urls=["https://www.billboard.com/charts/hot-100"]

## (1) Scrapping billboard charts

In [76]:
url=urls[0]

def billboard(url):
    
    #(1) Sending request
    response = requests.get(url)
    response.status_code # 200 status code means OK!

    soup = BeautifulSoup(response.content, "html.parser")
    
    #(2) Creating empty lists
    rank=[]
    track=[]
    artist=[]
    last_week = []
    peak_rank =[]
    weeks_on_charts =[]
    genre=[]
    source=[]
    date=[]

    #(3) Saving the information in lists
    for i in range(len(soup.select(".chart-element__information"))):
        
        #respectful scrapping
        from time import sleep
        wait_time = np.random.randint(1,4)
        sleep(wait_time)
        
        #(A) Getting str of song as a list
        append_str = soup.select(".chart-element__information")[i].get_text()
        append_list = append_str.split("\n")

        for append_str in append_list:
            append_str=(" ".join(re.findall(r"[A-Za-z0-9üäöÜÄÖß$()]*", append_str))).replace(" "," ").lower()
        #earsing featurs
        
        #(B) Appending to empty lists
        rank.append(i+1)
        track.append(append_list[1])
        artist.append(append_list[2])
        last_week.append(append_list[5])
        peak_rank.append(append_list[6])
        weeks_on_charts.append(append_list[7])
        genre.append("charts")
        source.append("billboard")
        date.append("2021-02")
        
    #(4) Saving information in a df
    charts = pd.DataFrame({"rank":rank,"track":track,
                       "artist":artist,"last_week":last_week,
                       "peak_rank": peak_rank,"weeks_on_charts":weeks_on_charts,
                        "genre":genre, "source":source, "date":date
                      }) 
    
    return charts

In [77]:
charts=billboard(url)
charts

,rank,track,artist,last_week,peak_rank,weeks_on_charts,genre,source,date
0,1,Drivers License,Olivia Rodrigo,1 Last Week,1 Peak Rank,5 Weeks on Chart,charts,billboard,2021-02
1,2,Up,Cardi B,- Last Week,2 Peak Rank,1 Weeks on Chart,charts,billboard,2021-02
2,3,Blinding Lights,The Weeknd,3 Last Week,1 Peak Rank,62 Weeks on Chart,charts,billboard,2021-02
3,4,Save Your Tears,The Weeknd,8 Last Week,4 Peak Rank,9 Weeks on Chart,charts,billboard,2021-02
4,5,Mood,24kGoldn Featuring iann dior,2 Last Week,1 Peak Rank,27 Weeks on Chart,charts,billboard,2021-02
...,...,...,...,...,...,...,...,...,...
95,96,Almost Maybes,Jordan Davis,99 Last Week,95 Peak Rank,3 Weeks on Chart,charts,billboard,2021-02
96,97,Masterpiece,DaBaby,- Last Week,55 Peak Rank,3 Weeks on Chart,charts,billboard,2021-02
97,98,Prisoner,Miley Cyrus Featuring Dua Lipa,97 Last Week,54 Peak Rank,12 Weeks on Chart,charts,billboard,2021-02
98,99,Skin,Sabrina Carpenter,89 Last Week,48 Peak Rank,3 Weeks on Chart,charts,billboard,2021-02


In [78]:
def billboard_cleaning(charts):
    pattern=re.compile("\d+")

    charts["weeks_on_charts"]=charts["weeks_on_charts"].apply(lambda x: int(pattern.search(x).group(0)))
    charts["peak_rank"]=charts["peak_rank"].apply(lambda x: int(pattern.search(x).group(0)))
    charts["last_week"]=list(map(lambda x: x.replace(" Last Week",""),charts["last_week"]))
    charts["last_week"]=list(map(lambda x: 0 if x== "-" else int(x),charts["last_week"]))
    
    return charts

In [79]:
charts=billboard_cleaning(charts)

In [80]:
#not necesarry for inital database

def add_new_songs(charts):
    
    new=charts[charts["last_week"]==0]
    new2=new.drop(["rank","last_week","peak_rank","weeks_on_charts"],axis=1)
    charts2=charts.drop(["rank","last_week","peak_rank","weeks_on_charts"],axis=1)
    charts_new=pd.concat([charts2,new2],axis=0,ignore_index=True)
    
    
    return charts_new

In [85]:
#charts_new=add_new_songs(charts)
charts_new.drop_duplicates()
charts_new

,track,artist,genre,source,date
0,Drivers License,Olivia Rodrigo,charts,billboard,2021-02
1,Up,Cardi B,charts,billboard,2021-02
2,Blinding Lights,The Weeknd,charts,billboard,2021-02
3,Save Your Tears,The Weeknd,charts,billboard,2021-02
4,Mood,24kGoldn Featuring iann dior,charts,billboard,2021-02
...,...,...,...,...,...
108,No More Parties,Coi Leray,charts,billboard,2021-02
109,Bandido,Myke Towers & Juhn,charts,billboard,2021-02
110,Lifestyle,Jason Derulo Featuring Adam Levine,charts,billboard,2021-02
111,Masterpiece,DaBaby,charts,billboard,2021-02


In [87]:
def save_df(charts_new):
    
    charts_new.to_csv("/Users/carolinvogt/Becoming_Data_Analyst/Week_07/Day_02/charts.csv")
    print("Sucessfully safed")
    
save_df(charts_new)

Sucessfully safed
